In [15]:
import snowflake.connector
import pandas as pd
from datetime import datetime
from zoomtools import solr
import random
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from sqlalchemy.dialects import registry
from snowflake.connector.pandas_tools import pd_writer # module not found
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

In [2]:
SF_URL = 'discoverorg.snowflakecomputing.com'
SF_ACCOUNT = 'discoverorg'

SF_USER = 'RWESTBROOK'
SF_ROLE = 'DATA_STRATEGY'

SF_WAREHOUSE = 'DATA_STRAT'
SF_DATABASE = 'SANDBOX_DATA_STRATEGY'
SF_SCHEMA = 'STAGING'

In [3]:
SF_Options= {
  'account' : SF_ACCOUNT,
  'user' : SF_USER,
  'warehouse' : SF_WAREHOUSE,
  'schema': SF_SCHEMA,
  'database': SF_DATABASE,
  'role': SF_ROLE,
  'authenticator': 'externalbrowser'
  }

In [7]:
ctx = snowflake.connector.connect(**SF_Options)

cur = ctx.cursor() 

In [5]:
query = """
SELECT INDIVIDUAL_ID, PRIMARY_TITLE
FROM PRODUCTION_DATA_DOZI.DOZI.PERSON
LIMIT 10
;
"""

In [10]:
cur = ctx.cursor().execute(query)
sample_query = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [11]:
sample_query

,INDIVIDUAL_ID,PRIMARY_TITLE
0,4022754854,None
1,4022785046,"Manager, Catering"
2,4023046227,E-Mobility Proposal Engineer
3,4023277989,Authorized Real Estate Advisor
4,4023761003,None
5,4023794807,Electro-Mechanical Engineering Technician
6,4023797158,None
7,4023850279,None
8,4023920967,"Postdoctoral Fellow, Pharmacometrics"
9,4024112905,French Teacher


In [16]:
# send this new data straight into Snowflake

sf = create_engine(URL(**SF_Options))
test = pd.DataFrame(sample_query) # replace sample_query with dataframe name
test.to_sql(name='test',con=sf, index=False, method=pd_writer) # name = table name to be written to Snowflake